# LangGraph Chatbot with Ollama

## Prerequisites
- Install required packages: `pip install langgraph langchain-community langchain-core`
- Install and run Ollama locally
- Pull the Gemma model: `ollama pull gemma:2b`

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_community.chat_models import ChatOllama
from typing import TypedDict, Annotated, Sequence
from operator import add
import json
import time

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[HumanMessage | AIMessage], add]

In [ ]:
llm_model = ChatOllama(model="gemma:2b", temperature=0.7)

sys_prompt = SystemMessage(
    content="You are Agent 007, a helpful and sophisticated AI assistant. "
            "Answer questions clearly, professionally, and with a touch of wit when appropriate."
)

In [ ]:
def llm_node(state: AgentState) -> AgentState:
    messages = [sys_prompt] + list(state["messages"])
    response = llm_model.invoke(messages)
    return {"messages": [response]}

In [ ]:
def create_agent():
    graph = StateGraph(AgentState)
    graph.add_node("llm_model", llm_node)
    graph.set_entry_point("llm_model")
    graph.add_edge("llm_model", END)
    return graph.compile()

bot = create_agent()

In [ ]:
def ask_bot(question: str, conversation_history=None):
    if conversation_history is None:
        conversation_history = []
    
    user_msg = HumanMessage(content=question)
    messages = conversation_history + [user_msg]
    
    result = bot.invoke({"messages": messages})
    return result["messages"][-1].content

In [ ]:
def run_interactive_chat():
    print("🤖 Agent 007 Interactive Chat")
    print("Type your messages below. Type 'exit', 'bye', or 'quit' to end the conversation.")
    print("=" * 60)
    
    history = []
    turn_count = 0

    while True:
        try:
            user_input = input(f"\n[Turn {turn_count + 1}] 👤 You: ").strip()
            
            if user_input.lower() in ["exit", "bye", "quit", "stop"]:
                print("\n🤖 Agent 007: Until we meet again. Stay safe out there! 🕴️")
                break
            
            if not user_input:
                print("Please enter a message or type 'exit' to quit.")
                continue

            user_msg = HumanMessage(content=user_input)
            history.append(user_msg)

            print("🤔 Agent 007 is thinking...")
            result = bot.invoke({"messages": history})
            ai_msg = result["messages"][-1]
            
            print(f"\n🤖 Agent 007: {ai_msg.content}")

            history.append(ai_msg)
            turn_count += 1
            
        except KeyboardInterrupt:
            print("\n\n🤖 Agent 007: Mission interrupted. Farewell! 🕴️")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            print("Please try again or type 'exit' to quit.")

In [ ]:
def analyze_conversation(history):
    if not history:
        print("No conversation history to analyze.")
        return
    
    human_messages = [msg for msg in history if isinstance(msg, HumanMessage)]
    ai_messages = [msg for msg in history if isinstance(msg, AIMessage)]
    
    print("📊 Conversation Analysis")
    print("=" * 25)
    print(f"Total messages: {len(history)}")
    print(f"Human messages: {len(human_messages)}")
    print(f"AI messages: {len(ai_messages)}")
    
    if human_messages:
        avg_human_length = sum(len(msg.content) for msg in human_messages) / len(human_messages)
        print(f"Average human message length: {avg_human_length:.1f} characters")
    
    if ai_messages:
        avg_ai_length = sum(len(msg.content) for msg in ai_messages) / len(ai_messages)
        print(f"Average AI message length: {avg_ai_length:.1f} characters")

def save_conversation(history, filename="conversation.json"):
    conversation_data = []
    
    for msg in history:
        msg_data = {
            "type": "human" if isinstance(msg, HumanMessage) else "ai",
            "content": msg.content,
            "timestamp": time.time()
        }
        conversation_data.append(msg_data)
    
    with open(filename, 'w') as f:
        json.dump(conversation_data, f, indent=2)
    
    print(f"💾 Conversation saved to {filename}")

def quick_chat(questions_list):
    history = []
    
    for i, question in enumerate(questions_list, 1):
        print(f"\n🔸 Question {i}: {question}")
        
        user_msg = HumanMessage(content=question)
        history.append(user_msg)
        
        result = bot.invoke({"messages": history})
        ai_response = result["messages"][-1]
        
        print(f"🤖 Response: {ai_response.content}")
        print("-" * 40)
        
        history.append(ai_response)
    
    return history

In [ ]:
# Usage examples:
# ask_bot("Your question here")
# run_interactive_chat()
# quick_chat(["question1", "question2", "question3"])
# analyze_conversation(history)
# save_conversation(history, "filename.json")